### Optimización de Hiperparámetros (Hyperparameter Tuning)

Comenzamos por crear un estimador de **Random Forest** para observar los valores predeterminados de los hiperparámetros disponibles.

Procedemos a imprimir los parámetros sobre los que vamos a hacer una búsqueda más adelante.

In [26]:
from pprint import pprint
pprint(clf.get_params())

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


### Tuning usando Random Search

Antes de buscar qué combinación de valores de parámetros conduce al modelo más preciso, debemos especificar los diferentes valores candidatos que queremos probar mediante un simple diccionario. En el siguiente código tenemos una serie de valores de parámetros candidatos, que incluyen varias posibilidades para los siguientes hiperparámetros:
The hyperparameters which I decided to focus on are:

1. n_estimators: número de estimadores en el Random Forest)
2. max_features: número máximo de atributos para la partición de nodos; generalmente < número de características en el dataset)
3. max_depth: número de niveles en cada Decision Tree)
4. min_samples_split: número mínimo de datos en un nodo antes de que este se parta hacia el siguiente nivel 
5. min_samples_leaf: número mínimo de datos en un único nodo 
5. criterion: métrica usada para fijar el `stopping criteria` de los árboles de decisión

Importamos `RandomizedSearchCV()` para, definidos los hiperparámetros, crear el estimador que iterará por todas las posibilidades facilitadas:

In [27]:
from sklearn.model_selection import RandomizedSearchCV

In [28]:
hyperparameters = {'max_features':[None, 'auto', 'sqrt', 'log2'],
                   'max_depth':[None, 1, 5, 10, 15, 20],
                   'min_samples_leaf': [1, 2, 4],
                   'min_samples_split': [2, 5, 10],
                   'n_estimators': [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)],
                   'criterion': ['gini', 'entropy']}

In [29]:
rf_random = RandomizedSearchCV(clf, hyperparameters, 
                               n_iter = 100, 
                               cv = 10, 
                               verbose=2, 
                               random_state=123, 
                               n_jobs = -1)

Para limitar la búsqueda, primero se ejecuta una validación cruzada de búsqueda aleatoria. Se realiza una búsqueda aleatoria de parámetros usando $k = 10$ veces la validación cruzada (`cv = 10`), en 100 combinaciones diferentes (n_iter = 100) y con todos los núcleos disponibles al mismo tiempo (n_jobs = -1). 
La Random Search selecciona una combinación de características al azar en lugar de iterar en cada combinación posible. Una mayor n_iter y cv dan como resultado más combinaciones y menos posibilidad de overfitting respectivamente.

Ejecutamos la búsqueda aleatoria mediante el entrenamiento de múltiples Random Forest con los parámetros anteriormente definidos permutados. En nuestro caso, la ejecución de esta línea de código tardó unos 9 minutos aproximadamente.

In [30]:
rf_random.fit(x_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  6.4min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'max_features': [None, 'auto', 'sqrt', 'log2'], 'max_depth': [None, 1, 5, 10, 15, 20], 'min_samples_leaf': [1, 2, 4], 'min_samples_split': [2, 5, 10], 'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'criterion': ['gini', 'entropy']},
          pre_dispatch='2*n_jobs', random_state=123, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

Mediante el siguiente comando podemos visualizar los mejores parámetros tras este proceso combinado de validación cruzada y finetuning de parámetros:

In [31]:
rf_random.best_params_

{'criterion': 'entropy',
 'max_depth': None,
 'max_features': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 80}